In [5]:
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import CSVLogger, WandbLogger, TensorBoardLogger
from pytorch_lightning.strategies.ddp import DDPStrategy
import sys
import os
import argparse
import logging
import yaml
logging.basicConfig(level=logging.INFO, format='%(levelname)s:%(message)s')
import wandb
import numpy as np
import torch

sys.path.append("/global/cfs/cdirs/m3443/usr/pmtuan/Tracking-ML-Exa.TrkX/")
from Pipelines.TrackML_Example_Dev.LightningModules.Embedding.Models.layerless_embedding import LayerlessEmbedding
from utils import headline
from datetime import datetime


In [19]:
from pytorch_lightning import Callback
class myLayerlessEmbedding(LayerlessEmbedding):

    def training_step_end(self, step_output):
        loss = step_output['loss']
        positive_loss = step_output['positive_loss'] 
        negative_loss = step_output['negative_loss']
        if self.trainer.current_epoch %2 == 0: 
            return negative_loss
        else:
            return positive_loss

class WeightRampCallback(Callback):

    def __init__(self):
        super().__init__()
        self.eff=[]
        self.weights=[]
    
    def on_validation_end(self, trainer, pl_module):
        self.eff.append(trainer.logged_metrics['eff'].cpu().numpy())
        eff_threshold = pl_module.hparams.get('eff_threshold', 0.8)
        patience = pl_module.hparams.get('patience', 10)
        weight_factor = pl_module.hparams.get('weight_factor', 1.1)
        if np.mean(self.eff[-10:]) < eff_threshold:
            self.weights.append(pl_module.hparams['weight'])
        elif np.mean(self.weights[ -patience : ]) == pl_module.hparams['weight']:
            pl_module.hparams['weight'] *= weight_factor
            self.weights.append(pl_module.hparams['weight'])
        else:
            self.weights.append(pl_module.hparams['weight'])


In [20]:
config_file = 'metric_learning_config_ITk_1.yaml'
with open(config_file) as file:
    all_configs = yaml.load(file, Loader=yaml.FullLoader)

common_configs = all_configs["common_configs"]
metric_learning_configs = all_configs["metric_learning_configs"]
metric_learning_configs['n_events'] = 10

model = myLayerlessEmbedding(metric_learning_configs)

trainer = Trainer(
    accelerator='gpu',
    devices=1,
    max_epochs=3,
    callbacks=[WeightRampCallback()]
)

trainer.fit(model)


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
INFO:Loaded event: /global/cfs/cdirs/m3443/data/ITk-upgrade/processed/feature_store/feature_store_modulewise_barrel_v3/10001
INFO:Loaded event: /global/cfs/cdirs/m3443/data/ITk-upgrade/processed/feature_store/feature_store_modulewise_barrel_v3/10002
INFO:Loaded event: /global/cfs/cdirs/m3443/data/ITk-upgrade/processed/feature_store/feature_store_modulewise_barrel_v3/10003
INFO:Loaded event: /global/cfs/cdirs/m3443/data/ITk-upgrade/processed/feature_store/feature_store_modulewise_barrel_v3/10004
INFO:Loaded event: /global/cfs/cdirs/m3443/data/ITk-upgrade/processed/feature_store/feature_store_modulewise_barrel_v3/10005
INFO:Loaded event: /global/cfs/cdirs/m3443/data/ITk-upgrade/processed/feature_store/feature_store_modulewise_barrel_v3/10006
INFO:Loaded event: /global/cfs/cdirs/m3443/data/ITk-upgrade/processed/feature_store/feature_store_modulew

/global/cfs/cdirs/m3443/usr/pmtuan/conda/exatrkx_hsf/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1933: PossibleUserWarning: The number of training batches (8) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Epoch 2: 100%|██████████| 9/9 [00:05<00:00,  1.63it/s, loss=0.00935, v_num=22, val_loss=0.00999, eff=0.234, pur=0.00045, current_lr=0.001, train_loss=0.0102, positive_loss=0.00023, negative_loss=0.00999, n_edges=7.17e+5, n_true_positive=1.54e+4, true_positive_rate=0.0215] 
